<a href="https://colab.research.google.com/github/Marqosneo/Stock-Analytics-ZoomCamp/blob/main/Module02_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0) Imports and Installs.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install yfinance
#!pip install pandas datareader

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"

response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

[      Filing Date Symbol                           Company Name  \
 0     May 3, 2024    TBN         Tamboran Resources Corporation   
 1    Apr 29, 2024   HWEC                   HW Electro Co., Ltd.   
 2    Apr 29, 2024   DTSQ  DT Cloud Star Acquisition Corporation   
 3    Apr 26, 2024   EURK                Eureka Acquisition Corp   
 4    Apr 26, 2024    HDL    Super Hi International Holding Ltd.   
 ..            ...    ...                                    ...   
 320  Jan 21, 2020   GOXS                            Goxus, Inc.   
 321  Jan 21, 2020   UTXO                 UTXO Acquisition, Inc.   
 322   Dec 9, 2019   LOHA                           Loha Co. Ltd   
 323   Oct 4, 2019   ZGHB  China Eco-Materials Group Co. Limited   
 324  Dec 27, 2018   FBOX              Fit Boxx Holdings Limited   
 
         Price Range Shares Offered  
 0                 -              -  
 1             $3.00        3750000  
 2            $10.00        6000000  
 3            $10.00        50

In [ ]:
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [ ]:
# convert to datetime
#ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'])
ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'], format='%b %d, %Y')

In [ ]:
# Filter by 2023 year
ipos_filings_2023 = ipos_filings[ipos_filings['Filing Date'].dt.year == 2023]

ipos_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,-
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
54,2023-12-22,LZMH,LZ Technology Holdings Limited,-,-
...,...,...,...,...,...
162,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000
163,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000
164,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,-
165,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000


In [ ]:
ipos_filings_2023.loc[:, ['Price Range', 'Shares Offered']] = ipos_filings_2023.loc[:, ['Price Range', 'Shares Offered']].replace('-', np.nan)

ipos_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
54,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN
...,...,...,...,...,...
162,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000
163,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000
164,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN
165,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000


In [ ]:
ipos_filings_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117 entries, 50 to 166
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     117 non-null    datetime64[ns]
 1   Symbol          117 non-null    object        
 2   Company Name    117 non-null    object        
 3   Price Range     90 non-null     object        
 4   Shares Offered  86 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 5.5+ KB


In [ ]:
import numpy as np

# Function to calculate average price
def calculate_avg_price(price_range):
    if isinstance(price_range, str) and '-' in price_range:
        # If a range is given, calculate the average
        prices = price_range.replace('$', '').split(' - ')
        return (float(prices[0].replace(',', '')) + float(prices[1].replace(',', ''))) / 2
    elif isinstance(price_range, str) and '-' not in price_range:
        # If only one price is provided, return that price
        return float(price_range.replace('$', '').replace(',', ''))
    else:
        # If no price is specified, return NaN
        return np.nan

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
ipos_filings_2023 = ipos_filings_2023.copy()

# Apply the function to create the 'Avg_price' column
ipos_filings_2023.loc[:, 'Avg_price'] = ipos_filings_2023['Price Range'].apply(calculate_avg_price)

# Display the DataFrame with the new 'Avg_price' column
ipos_filings_2023


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667,7.5
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000,4.5
54,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN
...,...,...,...,...,...,...
162,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000,4.5
163,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000,5.0
164,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN,NaN
165,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000,4.5


In [ ]:
import pandas as pd

# Function to calculate Shares_offered_value
def calculate_shares_offered_value(row):
    shares_offered = pd.to_numeric(row['Shares Offered'], errors='coerce')
    avg_price = pd.to_numeric(row['Avg_price'], errors='coerce')
    if not pd.isnull(shares_offered) and not pd.isnull(avg_price):
        return shares_offered * avg_price
    else:
        return np.nan

# Apply the function to create the 'Shares_offered_value' column
ipos_filings_2023['Shares_offered_value'] = ipos_filings_2023.apply(calculate_shares_offered_value, axis=1)

# Display the DataFrame with the new 'Shares_offered_value' column

ipos_filings_2023


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.0,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667,7.5,8000002.5
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000,4.5,9900000.0
54,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
162,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000,4.5,8437500.0
163,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000,5.0,16500000.0
164,2023-01-23,RPET,New Ruipeng Pet Group Inc.,NaN,NaN,NaN,NaN
165,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000,4.5,16875000.0


In [ ]:
# Filter the DataFrame for filings in 2023 and on Fridays
friday_filings_2023 = ipos_filings_2023[(ipos_filings_2023['Filing Date'].dt.year == 2023) & (ipos_filings_2023['Filing Date'].dt.dayofweek == 4)]

friday_filings_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.00,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN
54,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN,NaN
55,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN,NaN
60,2023-12-15,GIT,Going International Holding Company Limited,NaN,NaN,NaN,NaN
62,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000,5.00,10000000.0
63,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000,5.00,11000000.0
82,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000,4.50,6750000.0
89,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000,4.00,8000000.0
92,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000,4.00,8500000.0


In [ ]:
friday_filings_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 165
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           25 non-null     object        
 4   Shares Offered        25 non-null     object        
 5   Avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 2.0+ KB


In [ ]:
# Calculate the total sum in millions of USD
total_sum_millions_usd = friday_filings_2023['Shares_offered_value'].sum() / 1_000_000  # Divide by 1 million

# Round to the closest integer
total_sum_millions_usd = round(total_sum_millions_usd)

# Display the total sum
print("Total sum in $m (millions of USD) for all filings during 2023 on Fridays:", total_sum_millions_usd)


Total sum in $m (millions of USD) for all filings during 2023 on Fridays: 286


# Answer Question 1:
# Total sum in $m (millions of USD) for all filings during 2023 on Fridays: 286

Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

[         IPO Date Symbol                             Company Name IPO Price  \
 0    Dec 27, 2023   IROH            Iron Horse Acquisitions Corp.    $10.00   
 1    Dec 19, 2023   LGCB                       Linkage Global Inc     $4.00   
 2    Dec 15, 2023    ZKH                        ZKH Group Limited    $15.50   
 3    Dec 15, 2023   BAYA                 Bayview Acquisition Corp    $10.00   
 4    Dec 14, 2023   INHD                       Inno Holdings Inc.     $4.00   
 ..            ...    ...                                      ...       ...   
 149  Jan 25, 2023    QSG                     QuantaSing Group Ltd    $12.50   
 150  Jan 20, 2023   CVKD              Cadrenal Therapeutics, Inc.     $5.00   
 151  Jan 13, 2023   SKWD  Skyward Specialty Insurance Group, Inc.    $15.00   
 152  Jan 13, 2023   ISRL                 Israel Acquisitions Corp    $10.00   
 153  Jan 13, 2023   MGOL                          MGO Global Inc.     $5.00   
 
     Current   Return  
 0    $10.05  

In [ ]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [ ]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipo_dfs

[        IPO Date Symbol                                    Company Name  \
 0    May 8, 2024    NNE                        Nano Nuclear Energy Inc.   
 1    May 1, 2024    VIK                            Viking Holdings Ltd.   
 2   Apr 26, 2024   ZONE                       CleanCore Solutions, Inc.   
 3   Apr 25, 2024   RBRK                                    Rubrik, Inc.   
 4   Apr 25, 2024   LOAR                              Loar Holdings Inc.   
 ..           ...    ...                                             ...   
 60  Jan 18, 2024   PSBD                  Palmer Square Capital BDC Inc.   
 61  Jan 18, 2024   CCTG  CCSC Technology International Holdings Limited   
 62  Jan 12, 2024   SYNX                                  Silynxcom Ltd.   
 63  Jan 11, 2024   SDHC                       Smith Douglas Homes Corp.   
 64   Jan 9, 2024   ROMA                      Roma Green Finance Limited   
 
    IPO Price Current   Return  
 0      $4.00   $4.30   36.50%  
 1     $24.00  $28.4

In [ ]:

ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [ ]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$4.30,36.50%
1,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.44,19.38%
2,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.17,-21.25%
3,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$33.34,5.53%
4,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.94,77.96%
...,...,...,...,...,...,...
214,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.15,-74.48%
215,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.50,-90.40%
216,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.73,150.73%
217,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [ ]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$4.30,36.50%


In [ ]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      219 non-null    object
 1   Symbol        219 non-null    object
 2   Company Name  219 non-null    object
 3   IPO Price     219 non-null    object
 4   Current       219 non-null    object
 5   Return        219 non-null    object
dtypes: object(6)
memory usage: 10.4+ KB


In [ ]:
# Extract the "Company Name" column to a list
company_symbol_list = stacked_ipos_df['Symbol'].tolist()

# Display the list of company names
print(company_symbol_list)


['NNE', 'VIK', 'ZONE', 'RBRK', 'LOAR', 'MRX', 'NCI', 'YYGH', 'MFI', 'TRSG', 'IBTA', 'CTRI', 'MTEN', 'CDTG', 'JUNE', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'UBXG', 'MMA', 'IBAC', 'AUNA', 'RDDT', 'BKHA', 'LOBO', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'TBBB', 'MGX', 'HLXB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'BTSG', 'AVBP', 'HAO', 'CGON', 'YIBO', 'SUGP', 'JL', 'KSPI', 'JVSA', 'PSBD', 'CCTG', 'SYNX', 'SDHC', 'ROMA', 'IROH', 'LGCB', 'ZKH', 'BAYA', 'INHD', 'AFJK', 'GSIW', 'FEBO', 'CLBR', 'ELAB', 'RR', 'DDC', 'SHIM', 'GLAC', 'SGN', 'HG', 'CRGX', 'ANSC', 'AITR', 'GVH', 'LXEO', 'PAPL', 'ATGL', 'MNR', 'WBUY', 'NCL', 'BIRK', 'GMM', 'PMEC', 'LRHC', 'GPAK', 'SPKL', 'QETA', 'MSS', 'ANL', 'SYRA', 'VSME', 'LRE', 'TURB', 'MDBH', 'KVYO', 'CART', 'DTCK', 'RYZB', 'NMRA', 'ARM', 'SPPL', 'NWGL', 'SWIN', 'IVP', 'NNAG', 'SRM', 'SPGC', 'LQR', 'NRXS', 'FTEL', 'MIRA', 'PXDT', 'HRYU', 'CTNT', 'SRFM', 'PRZO', 'HYAC', 'KVAC', 

In [ ]:
# Remove 'RYZB' from the list of symbols
company_symbol_list.remove('RYZB')

In [ ]:
# Empty DataFrame to store OHLCV data
ohlc_data = pd.DataFrame()

# Iterate over the symbols and fetch OHLCV data
for symbol in company_symbol_list:
    try:
        # Fetch OHLCV data from Yahoo Finance
        ticker = yf.Ticker(symbol)
        ohlc = ticker.history(start='2023-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))  # Fetch data since 2023-01-01 to today
        ohlc['Symbol'] = symbol  # Add symbol column
        ohlc_data = pd.concat([ohlc_data, ohlc], axis=0)
    except Exception as e:
        print(f"Failed to fetch data for symbol {symbol}: {e}")

# Display the OHLCV data
ohlc_data


ERROR:yfinance:NNE: No price data found, symbol may be delisted (1d 2023-01-01 -> 2024-05-08)
ERROR:yfinance:BKHA: No timezone found, symbol may be delisted
ERROR:yfinance:PTHR: No timezone found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume,Symbol,Dividends,Stock Splits
Date,,,,,,,,,
2024-05-01 00:00:00-04:00,26.150000,26.860001,25.709999,26.100000,NaN,35017400.0,VIK,0.0,0.0
2024-05-02 00:00:00-04:00,26.540001,28.290001,26.315001,26.990000,NaN,10635500.0,VIK,0.0,0.0
2024-05-03 00:00:00-04:00,27.754999,29.270000,27.200001,29.000000,NaN,5908800.0,VIK,0.0,0.0
2024-05-06 00:00:00-04:00,29.080000,29.080000,27.980000,28.940001,NaN,3604500.0,VIK,0.0,0.0
2024-05-07 00:00:00-04:00,29.010000,29.455000,28.620001,28.650000,NaN,1743100.0,VIK,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2024-05-01 00:00:00-04:00,0.236000,0.252000,0.236000,0.250000,NaN,16000.0,MGOL,0.0,0.0
2024-05-02 00:00:00-04:00,0.250000,0.250000,0.228000,0.233000,NaN,118800.0,MGOL,0.0,0.0
2024-05-03 00:00:00-04:00,0.226000,0.244000,0.221000,0.232000,NaN,63300.0,MGOL,0.0,0.0


In [ ]:
# Remove 'BKHA' from the list of symbols
company_symbol_list.remove('BKHA')

# Remove 'PTHR' from the list of symbols
company_symbol_list.remove('PTHR')

# Remove 'NNE' from the list of symbols
company_symbol_list.remove('NNE')

In [ ]:
# Empty DataFrame to store OHLCV data
ohlc_data_ipos = pd.DataFrame()

# Iterate over the symbols and fetch OHLCV data
for symbol in company_symbol_list:
    try:
        # Fetch OHLCV data from Yahoo Finance
        ticker = yf.Ticker(symbol)
        ohlc = ticker.history(start='2023-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))  # Fetch data since 2023-01-01 to today
        ohlc['Symbol'] = symbol  # Add symbol column
        ohlc_data_ipos = pd.concat([ohlc_data_ipos, ohlc], axis=0)
    except Exception as e:
        print(f"Failed to fetch data for symbol {symbol}: {e}")

# Display the OHLCV data

ohlc_data_ipos

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2024-05-01 00:00:00-04:00,26.150000,26.860001,25.709999,26.100000,35017400,0.0,0.0,VIK
2024-05-02 00:00:00-04:00,26.540001,28.290001,26.315001,26.990000,10635500,0.0,0.0,VIK
2024-05-03 00:00:00-04:00,27.754999,29.270000,27.200001,29.000000,5908800,0.0,0.0,VIK
2024-05-06 00:00:00-04:00,29.080000,29.080000,27.980000,28.940001,3604500,0.0,0.0,VIK
2024-05-07 00:00:00-04:00,29.010000,29.455000,28.620001,28.650000,1743100,0.0,0.0,VIK
...,...,...,...,...,...,...,...,...
2024-05-01 00:00:00-04:00,0.236000,0.252000,0.236000,0.250000,16000,0.0,0.0,MGOL
2024-05-02 00:00:00-04:00,0.250000,0.250000,0.228000,0.233000,118800,0.0,0.0,MGOL
2024-05-03 00:00:00-04:00,0.226000,0.244000,0.221000,0.232000,63300,0.0,0.0,MGOL


In [ ]:
# Convert the index to datetime
ohlc_data_ipos.index = pd.to_datetime(ohlc_data_ipos.index)

# Filter the DataFrame
ohlc_data_ipos_fltered = ohlc_data_ipos[ohlc_data_ipos.index < '2024-03-01']

ohlc_data_ipos_fltered

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2024-02-27 00:00:00-05:00,3.500,9.350,3.500,8.000,2840400,0.0,0.0,SMXT
2024-02-28 00:00:00-05:00,6.420,7.240,5.600,5.860,243200,0.0,0.0,SMXT
2024-02-29 00:00:00-05:00,5.740,5.840,4.300,4.890,130000,0.0,0.0,SMXT
2024-02-26 00:00:00-05:00,1.390,1.800,1.130,1.635,2444600,0.0,0.0,VHAI
2024-02-27 00:00:00-05:00,2.230,2.710,1.470,1.720,10196900,0.0,0.0,VHAI
...,...,...,...,...,...,...,...,...
2024-02-23 00:00:00-05:00,0.380,0.390,0.370,0.385,138600,0.0,0.0,MGOL
2024-02-26 00:00:00-05:00,0.388,0.440,0.371,0.425,204700,0.0,0.0,MGOL
2024-02-27 00:00:00-05:00,0.433,0.440,0.414,0.430,103000,0.0,0.0,MGOL


In [ ]:
ohlc_data_ipos_fltered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25666 entries, 2024-02-27 00:00:00-05:00 to 2024-02-29 00:00:00-05:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          25666 non-null  float64
 1   High          25666 non-null  float64
 2   Low           25666 non-null  float64
 3   Close         25666 non-null  float64
 4   Volume        25666 non-null  int64  
 5   Dividends     25666 non-null  float64
 6   Stock Splits  25666 non-null  float64
 7   Symbol        25666 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 1.8+ MB


In [ ]:
# Calculating the number of unique symbols
num_unique_symbols = ohlc_data_ipos_fltered['Symbol'].nunique()

print("Number of unique symbols:", num_unique_symbols)


Number of unique symbols: 180


In [ ]:
# Create a copy of the DataFrame
ohlc_data_ipos_fltered = ohlc_data_ipos_fltered.copy()

# Calculate growth for each future day (1 to 30) using the copied DataFrame
for i in range(1, 31):
    ohlc_data_ipos_fltered.loc[:, f'growth_future_{i}d'] = (ohlc_data_ipos_fltered['Close'].shift(-i) - ohlc_data_ipos_fltered['Close']) / ohlc_data_ipos_fltered['Close']

ohlc_data_ipos_fltered

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,growth_future_1d,growth_future_2d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-27 00:00:00-05:00,3.500,9.350,3.500,8.000,2840400,0.0,0.0,SMXT,-0.267500,-0.388750,...,-0.695625,-0.585000,-0.638125,-0.589375,-0.630000,-0.645000,1.381250,1.443750,1.575000,1.662500
2024-02-28 00:00:00-05:00,6.420,7.240,5.600,5.860,243200,0.0,0.0,SMXT,-0.165529,-0.720990,...,-0.433447,-0.505973,-0.439420,-0.494881,-0.515358,2.250853,2.336177,2.515358,2.634812,2.486348
2024-02-29 00:00:00-05:00,5.740,5.840,4.300,4.890,130000,0.0,0.0,SMXT,-0.665644,-0.648262,...,-0.407975,-0.328221,-0.394683,-0.419223,2.895705,2.997955,3.212679,3.355828,3.177914,3.132924
2024-02-26 00:00:00-05:00,1.390,1.800,1.130,1.635,2444600,0.0,0.0,VHAI,0.051988,0.021407,...,1.009174,0.810398,0.737003,10.651376,10.957186,11.599389,12.027523,11.495413,11.360856,11.685015
2024-02-27 00:00:00-05:00,2.230,2.710,1.470,1.720,10196900,0.0,0.0,VHAI,-0.029070,-0.197674,...,0.720930,0.651163,10.075581,10.366278,10.976744,11.383720,10.877907,10.749999,11.058139,10.680232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23 00:00:00-05:00,0.380,0.390,0.370,0.385,138600,0.0,0.0,MGOL,0.103896,0.116883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-26 00:00:00-05:00,0.388,0.440,0.371,0.425,204700,0.0,0.0,MGOL,0.011765,-0.042353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-27 00:00:00-05:00,0.433,0.440,0.414,0.430,103000,0.0,0.0,MGOL,-0.053488,-0.146512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ohlc_data_ipos_fltered.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,growth_future_1d,growth_future_2d,growth_future_3d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,25666.000000,25666.000000,25666.000000,25666.000000,2.566600e+04,25666.000000,25666.000000,25665.000000,25664.000000,25663.000000,...,25645.000000,25644.000000,25643.000000,25642.000000,25641.000000,25640.000000,25639.000000,25638.000000,25637.000000,25636.000000
mean,13.069908,13.891856,12.472481,13.034087,7.043232e+05,0.000206,0.000784,0.077740,0.124563,0.164964,...,0.796963,0.832113,0.863411,0.893780,0.928038,0.958490,0.989302,1.017531,1.047315,1.079454
std,45.178041,68.234459,38.699050,48.434733,5.130104e+06,0.009609,0.124841,5.494531,5.872124,5.841310,...,7.683365,7.981812,8.092713,8.162951,8.464313,8.554822,8.562163,8.437432,8.451286,8.578700
min,0.107000,0.107000,0.095000,0.096000,0.000000e+00,0.000000,0.000000,-0.965258,-0.966747,-0.965329,...,-0.974610,-0.974780,-0.974383,-0.974474,-0.974180,-0.975195,-0.975759,-0.976755,-0.978605,-0.978086
25%,1.860000,1.970000,1.750000,1.850000,1.220000e+04,0.000000,0.000000,-0.025694,-0.037978,-0.048254,...,-0.199125,-0.206492,-0.215303,-0.221851,-0.227922,-0.234783,-0.240936,-0.247781,-0.254394,-0.261955
50%,7.000000,7.380000,6.640000,7.000000,6.370000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.000467,0.000000,-0.000973,-0.001613,-0.001905,-0.002134,-0.002892
75%,11.200000,11.750000,10.700000,11.150000,2.527750e+05,0.000000,0.000000,0.017182,0.023452,0.029854,...,0.107781,0.111820,0.118634,0.122644,0.127863,0.131939,0.137304,0.141171,0.147016,0.154509
max,3069.000000,7500.000000,875.000000,4318.000000,3.723413e+08,0.950000,20.000000,818.354842,824.621412,774.781533,...,697.705521,749.956522,752.577659,741.690042,784.090909,784.090909,757.875212,697.705521,682.011702,684.723368


In [ ]:
# Group by X days and calculate the 75th percentile growth
percentile_75 = {}
for i in range(1, 31):
    percentile_75[i] = ohlc_data_ipos_fltered[f'growth_future_{i}d'].quantile(0.75)

# Find the X with the highest 75th percentile growth
optimal_X = max(percentile_75, key=percentile_75.get)

print("Optimal X:", optimal_X)

Optimal X: 30


# Answer Question 2:
# Optimal X: 30

In [ ]:
# Create a copy of the DataFrame
ohlc_data_ipos_fltered_test = ohlc_data_ipos_fltered.copy()

# Calculate growth for each future day (1 to 30) using the copied DataFrame
for i in range(1, 41):
    ohlc_data_ipos_fltered_test.loc[:, f'growth_future_{i}d'] = (ohlc_data_ipos_fltered_test['Close'].shift(-i) - ohlc_data_ipos_fltered_test['Close']) / ohlc_data_ipos_fltered_test['Close']

ohlc_data_ipos_fltered_test

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,growth_future_1d,growth_future_2d,...,growth_future_31d,growth_future_32d,growth_future_33d,growth_future_34d,growth_future_35d,growth_future_36d,growth_future_37d,growth_future_38d,growth_future_39d,growth_future_40d
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-27 00:00:00-05:00,3.500,9.350,3.500,8.000,2840400,0.0,0.0,SMXT,-0.267500,-0.388750,...,1.553750,1.526250,1.592500,1.511250,1.598750,1.608750,1.586250,1.523750,1.612500,1.623750
2024-02-28 00:00:00-05:00,6.420,7.240,5.600,5.860,243200,0.0,0.0,SMXT,-0.165529,-0.720990,...,2.448805,2.539249,2.428328,2.547782,2.561434,2.530717,2.445393,2.566553,2.581911,0.759386
2024-02-29 00:00:00-05:00,5.740,5.840,4.300,4.890,130000,0.0,0.0,SMXT,-0.665644,-0.648262,...,3.241309,3.108385,3.251534,3.267894,3.231084,3.128835,3.274029,3.292434,1.108385,1.304704
2024-02-26 00:00:00-05:00,1.390,1.800,1.130,1.635,2444600,0.0,0.0,VHAI,0.051988,0.021407,...,11.287462,11.715597,11.764527,11.654435,11.348624,11.782874,11.837920,5.305811,5.892967,6.149847
2024-02-27 00:00:00-05:00,2.230,2.710,1.470,1.720,10196900,0.0,0.0,VHAI,-0.029070,-0.197674,...,11.087210,11.133721,11.029070,10.738372,11.151162,11.203488,4.994186,5.552326,5.796511,6.122093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23 00:00:00-05:00,0.380,0.390,0.370,0.385,138600,0.0,0.0,MGOL,0.103896,0.116883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-26 00:00:00-05:00,0.388,0.440,0.371,0.425,204700,0.0,0.0,MGOL,0.011765,-0.042353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-27 00:00:00-05:00,0.433,0.440,0.414,0.430,103000,0.0,0.0,MGOL,-0.053488,-0.146512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ohlc_data_ipos_fltered_test.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,growth_future_1d,growth_future_2d,growth_future_3d,...,growth_future_31d,growth_future_32d,growth_future_33d,growth_future_34d,growth_future_35d,growth_future_36d,growth_future_37d,growth_future_38d,growth_future_39d,growth_future_40d
count,25666.000000,25666.000000,25666.000000,25666.000000,2.566600e+04,25666.000000,25666.000000,25665.000000,25664.000000,25663.000000,...,25635.000000,25634.000000,25633.000000,25632.000000,25631.000000,25630.000000,25629.000000,25628.000000,25627.000000,25626.000000
mean,13.069908,13.891856,12.472481,13.034087,7.043232e+05,0.000206,0.000784,0.077740,0.124563,0.164964,...,1.111565,1.142002,1.172989,1.204306,1.233998,1.262328,1.289713,1.315888,1.342017,1.368781
std,45.178041,68.234459,38.699050,48.434733,5.130104e+06,0.009609,0.124841,5.494531,5.872124,5.841310,...,8.566656,8.596216,8.681190,8.794377,8.883447,8.919466,8.979150,9.002151,9.031368,9.097813
min,0.107000,0.107000,0.095000,0.096000,0.000000e+00,0.000000,0.000000,-0.965258,-0.966747,-0.965329,...,-0.977607,-0.978054,-0.979101,-0.980442,-0.980476,-0.981448,-0.980913,-0.980247,-0.979469,-0.980768
25%,1.860000,1.970000,1.750000,1.850000,1.220000e+04,0.000000,0.000000,-0.025694,-0.037978,-0.048254,...,-0.267966,-0.272315,-0.279383,-0.284931,-0.290489,-0.297288,-0.303571,-0.311019,-0.317397,-0.321839
50%,7.000000,7.380000,6.640000,7.000000,6.370000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.003395,-0.003256,-0.003316,-0.003953,-0.004748,-0.004739,-0.004348,-0.004125,-0.004283,-0.005171
75%,11.200000,11.750000,10.700000,11.150000,2.527750e+05,0.000000,0.000000,0.017182,0.023452,0.029854,...,0.158723,0.166558,0.171429,0.177029,0.182419,0.187921,0.192437,0.199060,0.204812,0.209194
max,3069.000000,7500.000000,875.000000,4318.000000,3.723413e+08,0.950000,20.000000,818.354842,824.621412,774.781533,...,638.703704,615.857143,611.048208,615.857143,615.857143,609.922481,615.857143,615.857143,607.169022,615.857143


In [ ]:
# Group by X days and calculate the 75th percentile growth
percentile_75 = {}
for i in range(1, 41):
    percentile_75[i] = ohlc_data_ipos_fltered_test[f'growth_future_{i}d'].quantile(0.75)

# Find the X with the highest 75th percentile growth
optimal_X = max(percentile_75, key=percentile_75.get)

print("Optimal X:", optimal_X)

Optimal X: 40


# Do you suggest pursuing this strategy for an optimal X?

# It is not a valid way to find the best number of days to invest as the 75th percentile is always increasing. A test has been done by calculating the 40-day return and again the higher value of days is the optimal one, something that does not even remotely resemble the graph seen in class 2. ("Reddit's (RDDT) price after the IPO")

Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [ ]:
LARGEST_STOCKS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
LARGEST_STOCKS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [ ]:
# Create an empty DataFrame to store the results
largest_stocks_df = pd.DataFrame()

# Define the date range with additional periods
start_date = '2013-12-18'  # 7 periods before 2014-01-01
end_date = '2023-12-31'

for i, ticker in enumerate(LARGEST_STOCKS):
    print(i, ticker)

    # Work with stock prices
    historyPrices = yf.download(tickers=ticker,
                                start=start_date,
                                end=end_date,
                                interval="1d")

    historyPrices['Ticker'] = ticker
    historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)

    # Filter data to the required date range (2014-01-01 to 2023-12-31)
    historyPrices = historyPrices['2014-01-01':'2023-12-31']

    # Append data to the DataFrame
    largest_stocks_df = pd.concat([largest_stocks_df, historyPrices], axis=0)

# Display the DataFrame
largest_stocks_df


0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

30 ITC.NS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS
32 LT.NS


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2014-01-02,37.349998,37.400002,37.099998,37.160000,31.233059,30632200,MSFT,1.009782
2014-01-03,37.200001,37.220001,36.599998,36.910000,31.022942,31134800,MSFT,1.007919
2014-01-06,36.849998,36.889999,36.110001,36.130001,30.367348,43603700,MSFT,0.974380
2014-01-07,36.330002,36.490002,36.209999,36.410000,30.602690,35802800,MSFT,0.972490
2014-01-08,36.000000,36.139999,35.580002,35.759998,30.056360,59971700,MSFT,0.958970
...,...,...,...,...,...,...,...,...
2023-12-22,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,1.022987
2023-12-26,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,1.016588
2023-12-27,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,1.016055


In [ ]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_US + NEW_EU + NEW_INDIA

LARGE_STOCKS

['TSLA',
 'WMT',
 'XOM',
 'UNH',
 'MA',
 'PG',
 'JNJ',
 'MRK',
 'HD',
 'COST',
 'ORCL',
 'PRX.AS',
 'CDI.PA',
 'AIR.PA',
 'SU.PA',
 'ETN',
 'SNY',
 'BUD',
 'DTE.DE',
 'ALV.DE',
 'MDT',
 'AI.PA',
 'EL.PA',
 'BAJFINANCE.NS',
 'MARUTI.NS',
 'HCLTECH.NS',
 'TATAMOTORS.NS',
 'SUNPHARMA.NS',
 'ONGC.NS',
 'ADANIENT.NS',
 'ADANIENT.NS',
 'NTPC.NS',
 'KOTAKBANK.NS',
 'TITAN.NS']

In [ ]:
# Create an empty DataFrame to store the results
large_stocks_df = pd.DataFrame()

# Define the date range with additional periods
start_date = '2013-12-18'  # 7 periods before 2014-01-01
end_date = '2023-12-31'

for i, ticker in enumerate(LARGE_STOCKS):
    print(i, ticker)

    # Work with stock prices
    historyPrices = yf.download(tickers=ticker,
                                start=start_date,
                                end=end_date,
                                interval="1d")

    historyPrices['Ticker'] = ticker
    historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)

    # Filter data to the required date range (2014-01-01 to 2023-12-31)
    historyPrices = historyPrices['2014-01-01':'2023-12-31']

    # Append data to the DataFrame
    large_stocks_df = pd.concat([large_stocks_df, historyPrices], axis=0)

# Display the DataFrame
large_stocks_df

[*********************100%%**********************]  1 of 1 completed

0 TSLA
1 WMT



[*********************100%%**********************]  1 of 1 completed


2 XOM


[*********************100%%**********************]  1 of 1 completed


...

32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


33 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2014-01-02,9.986667,10.165333,9.770000,10.006667,10.006667,92826000,TSLA,1.047892
2014-01-03,10.000000,10.146000,9.906667,9.970667,9.970667,70425000,TSLA,1.041867
2014-01-06,10.000000,10.026667,9.682667,9.800000,9.800000,80416500,TSLA,0.970874
2014-01-07,9.841333,10.026667,9.683333,9.957333,9.957333,75511500,TSLA,0.960514
2014-01-08,9.923333,10.246667,9.917333,10.085333,10.085333,92448000,TSLA,1.001059
...,...,...,...,...,...,...,...,...
2023-12-22,3580.000000,3638.449951,3560.550049,3627.350098,3627.350098,777099,TITAN.NS,1.006940
2023-12-26,3635.000000,3665.000000,3623.449951,3656.699951,3656.699951,526101,TITAN.NS,1.018182
2023-12-27,3668.000000,3695.000000,3645.000000,3689.250000,3689.250000,666625,TITAN.NS,1.024635


In [ ]:
largest_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80762 entries, 2014-01-02 to 2023-12-29
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       80762 non-null  float64
 1   High       80762 non-null  float64
 2   Low        80762 non-null  float64
 3   Close      80762 non-null  float64
 4   Adj Close  80762 non-null  float64
 5   Volume     80762 non-null  int64  
 6   Ticker     80762 non-null  object 
 7   growth_7d  80754 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.5+ MB


In [ ]:
# Filter the DataFrame to show rows where growth_7d is null
null_growth_largets = largest_stocks_df[largest_stocks_df['growth_7d'].isnull()]

# Display the DataFrame
null_growth_largets


,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2014-01-02,90.416405,90.957275,89.208443,89.235489,61.293831,3186241,SIE.DE,NaN
2022-05-17,872.000000,918.950012,860.000000,875.250000,865.802429,48792877,LICI.NS,NaN
2022-05-18,885.549988,891.000000,874.099976,876.349976,866.890564,8919761,LICI.NS,NaN
2022-05-19,867.000000,867.000000,838.000000,840.849976,831.773743,7372427,LICI.NS,NaN
2022-05-20,848.549988,856.799988,825.000000,826.150024,817.232422,7078291,LICI.NS,NaN
2022-05-23,826.150024,852.700012,803.650024,816.849976,808.032776,12444648,LICI.NS,NaN
2022-05-24,818.000000,834.700012,816.849976,823.099976,814.215332,4287864,LICI.NS,NaN
2022-05-25,831.500000,841.450012,814.250000,820.299988,811.445557,4013031,LICI.NS,NaN


In [ ]:
large_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 83845 entries, 2014-01-02 to 2023-12-29
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       83845 non-null  float64
 1   High       83845 non-null  float64
 2   Low        83845 non-null  float64
 3   Close      83845 non-null  float64
 4   Adj Close  83845 non-null  float64
 5   Volume     83845 non-null  int64  
 6   Ticker     83845 non-null  object 
 7   growth_7d  83836 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.8+ MB


In [ ]:
# Filter the DataFrame to show rows where growth_7d is null
null_growth_large = large_stocks_df[large_stocks_df['growth_7d'].isnull()]

# Display the DataFrame
null_growth_large

,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2019-09-11,34.868782,35.511101,33.207928,34.038357,33.763878,14787563,PRX.AS,NaN
2019-09-12,34.501743,34.524685,32.978527,33.148285,32.880985,5842906,PRX.AS,NaN
2019-09-13,33.171223,34.042942,32.804184,33.905304,33.631897,5068221,PRX.AS,NaN
2019-09-16,33.515324,33.721783,32.340797,32.340797,32.080013,49689045,PRX.AS,NaN
2019-09-17,32.758305,33.033585,31.574600,31.794825,31.538439,10523710,PRX.AS,NaN
2019-09-18,31.638832,31.748943,30.868050,31.060745,30.810278,6887058,PRX.AS,NaN
2019-09-19,31.290144,31.790236,30.657001,31.565424,31.310890,6470299,PRX.AS,NaN
2014-01-02,12.460000,12.555000,12.120000,12.130000,8.384733,14491222,DTE.DE,NaN
2014-01-02,130.949997,131.399994,127.849998,128.149994,79.434074,1524683,ALV.DE,NaN


In [ ]:
# nulls are eliminated
largest_stocks_df = largest_stocks_df.dropna(subset=['growth_7d'])
large_stocks_df = large_stocks_df.dropna(subset=['growth_7d'])

In [ ]:
largest_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80754 entries, 2014-01-02 to 2023-12-29
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       80754 non-null  float64
 1   High       80754 non-null  float64
 2   Low        80754 non-null  float64
 3   Close      80754 non-null  float64
 4   Adj Close  80754 non-null  float64
 5   Volume     80754 non-null  int64  
 6   Ticker     80754 non-null  object 
 7   growth_7d  80754 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.5+ MB


In [ ]:
large_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 83836 entries, 2014-01-02 to 2023-12-29
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       83836 non-null  float64
 1   High       83836 non-null  float64
 2   Low        83836 non-null  float64
 3   Close      83836 non-null  float64
 4   Adj Close  83836 non-null  float64
 5   Volume     83836 non-null  int64  
 6   Ticker     83836 non-null  object 
 7   growth_7d  83836 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.8+ MB


In [ ]:
average_daily_growth_largest = largest_stocks_df.groupby('Date')['growth_7d'].mean().reset_index()
average_daily_growth_largest['Category'] = 'LARGEST'
average_daily_growth_largest.columns = ['Date', 'Avg_7d', 'Category']
average_daily_growth_largest

,Date,Avg_7d,Category
0,2014-01-01,1.011797,LARGEST
1,2014-01-02,1.005147,LARGEST
2,2014-01-03,0.999152,LARGEST
3,2014-01-06,0.994203,LARGEST
4,2014-01-07,0.992509,LARGEST
...,...,...,...
2590,2023-12-22,1.013788,LARGEST
2591,2023-12-26,1.014900,LARGEST
2592,2023-12-27,1.014155,LARGEST
2593,2023-12-28,1.008533,LARGEST


In [ ]:
average_daily_growth_large = large_stocks_df.groupby('Date')['growth_7d'].mean().reset_index()
average_daily_growth_large['Category'] = 'LARGE'
average_daily_growth_large.columns = ['Date', 'Avg_7d', 'Category']
average_daily_growth_large

,Date,Avg_7d,Category
0,2014-01-01,1.011684,LARGE
1,2014-01-02,1.000970,LARGE
2,2014-01-03,1.000875,LARGE
3,2014-01-06,0.995983,LARGE
4,2014-01-07,0.991592,LARGE
...,...,...,...
2590,2023-12-22,1.003014,LARGE
2591,2023-12-26,1.013505,LARGE
2592,2023-12-27,1.002167,LARGE
2593,2023-12-28,0.998583,LARGE


In [ ]:
# Merge the two DataFrames on the 'Date' column
merged_df = average_daily_growth_largest.merge(average_daily_growth_large, on='Date', suffixes=('_largest', '_large'))

# Calculate the number of days when the LARGE group outperforms the LARGEST group
outperform_days = (merged_df['Avg_7d_large'] > merged_df['Avg_7d_largest']).sum()

# Calculate percentage
total_days = 2595
percentage = (outperform_days / total_days) * 100

# Convert it to the closest integer value
percentage = round(percentage)

print(f"The percentage of days when the LARGE group outperformed the LARGEST group is approximately {percentage}%.")


The percentage of days when the LARGE group outperformed the LARGEST group is approximately 47%.


# Answer Question 3:
# The percentage of days when the LARGE group outperformed the LARGEST group is approximately 47%.

Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS Dolars (closest integer value) over many operations in 10 years? One operation calculations: if you invested Dolar 1000 and received Dolar 1010 in 5 days - you add Dolar 10 to gross profit, if you received Dolar980 - add -Dolar20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [ ]:
# Load the DataFrame
stocks_df_combined = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_2024_05_07.parquet.brotli")

# Filter the DataFrame to the last 10 years of data
start_date = "2014-01-01"
end_date = "2023-12-31"
stocks_df_truncated = stocks_df_combined[(stocks_df_combined['Date'] >= start_date) & (stocks_df_combined['Date'] <= end_date)]

# Save the truncated DataFrame
stocks_df_truncated.to_parquet("/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/my_stocks_df_combined_trunc_2014_2023.parquet.brotli", compression="brotli")


In [ ]:
import pandas as pd

# Load both files into DataFrames
file1_path = "/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_trunc_2014_2023.parquet.brotli"
file2_path = "/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/my_stocks_df_combined_trunc_2014_2023.parquet.brotli"

df1 = pd.read_parquet(file1_path)
df2 = pd.read_parquet(file2_path)

# Compare the DataFrames
differences = df1.compare(df2)

if differences.empty:
    print("The files have identical content.")
else:
    print("The files differ in the following ways:")
    print(differences)


The files differ in the following ways:
     Adj Close_x            growth_1d           growth_3d           growth_7d  \
            self      other      self     other      self     other      self   
7011   31.233059  31.233065  0.993317  0.993318  0.996513  0.996514  1.009782   
7012   31.022930  31.022938  0.993272  0.993272  0.989809  0.989810  1.007919   
7013   30.367352  30.367342  0.978868  0.978867  0.965785  0.965785  0.974380   
7014   30.602673  30.602690  1.007749  1.007750  0.979817  0.979817  0.972489   
7015   30.056356  30.056362  0.982148  0.982148  0.968843  0.968843  0.958970   
...          ...        ...       ...       ...       ...       ...       ...   
5338         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
5339         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
5340         NaN        NaN       NaN       NaN       NaN       NaN       NaN   
5341         NaN        NaN       NaN       NaN       NaN       NaN  

I have differences between the file I truncated and the one offered for download. I will use the one offered in the download link so as not to give erroneous results.

In [ ]:
df_trunc = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_trunc_2014_2023.parquet.brotli')

df_trunc

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [ ]:
df_trunc.columns.to_list()

['Open',
 'High',
 'Low',
 'Close',
 'Adj Close_x',
 'Volume',
 'Ticker',
 'Year',
 'Month',
 'Weekday',
 'Date',
 'growth_1d',
 'growth_3d',
 'growth_7d',
 'growth_30d',
 'growth_90d',
 'growth_365d',
 'growth_future_5d',
 'SMA10',
 'SMA20',
 'growing_moving_average',
 'high_minus_low_relative',
 'volatility',
 'is_positive_growth_5d_future',
 'ticker_type',
 'index_x',
 'adx',
 'adxr',
 'apo',
 'aroon_1',
 'aroon_2',
 'aroonosc',
 'bop',
 'cci',
 'cmo',
 'dx',
 'macd',
 'macdsignal',
 'macdhist',
 'macd_ext',
 'macdsignal_ext',
 'macdhist_ext',
 'macd_fix',
 'macdsignal_fix',
 'macdhist_fix',
 'mfi',
 'minus_di',
 'mom',
 'plus_di',
 'dm',
 'ppo',
 'roc',
 'rocp',
 'rocr',
 'rocr100',
 'rsi',
 'slowk',
 'slowd',
 'fastk',
 'fastd',
 'fastk_rsi',
 'fastd_rsi',
 'trix',
 'ultosc',
 'willr',
 'index_y',
 'ad',
 'adosc',
 'obv',
 'atr',
 'natr',
 'ht_dcperiod',
 'ht_dcphase',
 'ht_phasor_inphase',
 'ht_phasor_quadrature',
 'ht_sine_sine',
 'ht_sine_leadsine',
 'ht_trendmod',
 'avgprice',

In [ ]:
# We create a smaller df for faster calculations
df_cci = df_trunc.loc[:, ['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker','Date', 'Year', 'Month', 'Weekday', 'cci', 'growth_future_5d']]
df_cci

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014-01-02,2014,2014-01-01,3,57.700615,0.956136
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014-01-03,2014,2014-01-01,4,1.373763,0.976429
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014-01-06,2014,2014-01-01,0,-96.631259,0.968170
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014-01-07,2014,2014-01-01,1,-83.904297,0.982698
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014-01-08,2014,2014-01-01,2,-147.855135,1.027964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023-12-22,2023,2023-12-01,4,70.767162,1.013657
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023-12-26,2023,2023-12-01,1,99.598220,0.985115
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023-12-27,2023,2023-12-01,2,130.401152,0.970753
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023-12-28,2023,2023-12-01,3,106.774509,0.983130


In [ ]:
# Filter records where CCI > 200 and it's a Friday
df_cci_filter = df_cci[(df_cci['cci'] > 200) & (df_cci['Weekday'] == 4)]

df_cci_filter

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395


In [ ]:
df_cci_filter['gross_profit'] = 1000 * df_cci_filter['growth_future_5d'] - 1000

df_cci_filter

<ipython-input-339-2fcaa4a6cc69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter['gross_profit'] = 1000 * df_cci_filter['growth_future_5d'] - 1000


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,gross_profit
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190,13.190094
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207,17.207263
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503,16.503151
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794,33.793947
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650,-4.350092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029,11.028926
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417,-10.583434
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476,24.475647
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395,2.395213


In [ ]:
# Calculate the cumulative gross profit
df_cci_filter['gross_profit_cum'] = df_cci_filter['gross_profit'].cumsum()

df_cci_filter

<ipython-input-340-d95a2c88b851>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter['gross_profit_cum'] = df_cci_filter['gross_profit'].cumsum()


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,gross_profit,gross_profit_cum
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190,13.190094,13.190094
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207,17.207263,30.397358
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503,16.503151,46.900509
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794,33.793947,80.694456
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650,-4.350092,76.344364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029,11.028926,973.115968
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417,-10.583434,962.532535
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476,24.475647,987.008181
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395,2.395213,989.403395


In [ ]:
print(f"The expected gross profit (no fees) over many operations in 10 years is {int(round((df_cci_filter['gross_profit_cum'].iloc[-1] / 1000), 1))} thousnad")


The expected gross profit (no fees) over many operations in 10 years is 1 thousnad


# Answer Question 4:
# The expected gross profit (no fees) over many operations in 10 years is 1 thousnad

# Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?


Total costs from Degiro

€2,210.50

# With the trading costs to be paid at Degiro. The strategy would not be profitable

# I have a problem

### If I do the procedure using the variable grow_future_5d given in the zip file, I get results like the ones requested in the questionnaire.
### If I try to calculate the gain by my own instructions I don't get the same result. I did it in three different ways and in all three I get a gross profit of 162,95

In [ ]:
df_trunc_1 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_trunc_2014_2023.parquet.brotli')

df_trunc_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [ ]:
# We create a smaller df for faster calculations
df_cci_1 = df_trunc_1.loc[:, ['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker','Date', 'Year', 'Month', 'Weekday', 'cci']]
df_cci_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014-01-02,2014,2014-01-01,3,57.700615
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014-01-03,2014,2014-01-01,4,1.373763
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014-01-06,2014,2014-01-01,0,-96.631259
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014-01-07,2014,2014-01-01,1,-83.904297
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014-01-08,2014,2014-01-01,2,-147.855135
...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023-12-22,2023,2023-12-01,4,70.767162
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023-12-26,2023,2023-12-01,1,99.598220
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023-12-27,2023,2023-12-01,2,130.401152
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023-12-28,2023,2023-12-01,3,106.774509


In [ ]:
# Filter records where CCI > 200 and it's a Friday
df_cci_filter_open_1 = df_cci_1[(df_cci_1['cci'] > 200) & (df_cci_1['Weekday'] == 4)]

df_cci_filter_open_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024
...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710


In [ ]:
df_cci_filter_open_1['stocks_buy'] = 1000 / df_cci_filter_open_1['Adj Close_x']

df_cci_filter_open_1

<ipython-input-368-d07adfad6050>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter_open_1['stocks_buy'] = 1000 / df_cci_filter_open_1['Adj Close_x'].copy()


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637


In [ ]:
# Step 1: Define an empty list to hold rows
rows = []

# Step 2: Iterate over each row in df_cci_filter_open
for index, row in df_cci_filter_open_1.iterrows():
    # Step 3: Find the corresponding row in df_cci using shift(-5)
    close_row = df_cci.shift(-5).loc[(df_cci['Date'] == row['Date']) & (df_cci['Ticker'] == row['Ticker'])]

    # Step 4: Append the found row to the list
    if not close_row.empty:
        rows.append(close_row.iloc[0])  # Append the first row if multiple rows are found

# Step 5: Concatenate the rows into df_cci_close_1
df_cci_close_1 = pd.concat(rows, axis=1).T

# Display df_cci_close_1
df_cci_close_1


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d
7113,41.48,41.66,41.240002,41.48,35.373264,24060500.0,MSFT,2014-06-06,2014.0,2014-06-01,4.0,192.611211,0.993973
7181,46.91,47.02,46.599998,46.700001,40.073509,38244700.0,MSFT,2014-09-12,2014.0,2014-09-01,4.0,101.585057,1.017559
7340,48.580002,48.880001,48.400002,48.66,42.31778,38937300.0,MSFT,2015-05-01,2015.0,2015-05-01,4.0,83.998285,0.981299
7452,47.450001,47.540001,46.919998,47.110001,41.507988,28600600.0,MSFT,2015-10-09,2015.0,2015-10-01,4.0,111.259873,1.008491
7467,53.32,53.990002,52.619999,52.639999,46.380405,46619800.0,MSFT,2015-10-30,2015.0,2015-10-01,4.0,69.356892,1.043313
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2096.100098,2098.899902,2069.149902,2085.5,2080.807617,1957264.0,LT.NS,2022-12-02,2022.0,2022-12-01,4.0,105.598993,1.03287
5219,2482.75,2504.0,2444.600098,2449.350098,2443.839111,1837661.0,LT.NS,2023-07-07,2023.0,2023-07-01,4.0,72.839846,1.009349
5234,2660.149902,2668.149902,2625.0,2649.550049,2643.588623,1325724.0,LT.NS,2023-07-28,2023.0,2023-07-01,4.0,107.527599,0.993687
5268,2933.0,2934.949951,2900.0,2908.550049,2908.550049,1655275.0,LT.NS,2023-09-15,2023.0,2023-09-01,4.0,74.635302,0.99792


In [ ]:
# Add a new column "open_adj_close_x" to df_cci_close_1
df_cci_close_1['stocks_buy'] = df_cci_filter_open_1['stocks_buy']

df_cci_close_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,stocks_buy
7113,41.48,41.66,41.240002,41.48,35.373264,24060500.0,MSFT,2014-06-06,2014.0,2014-06-01,4.0,192.611211,0.993973,28.642821
7181,46.91,47.02,46.599998,46.700001,40.073509,38244700.0,MSFT,2014-09-12,2014.0,2014-09-01,4.0,101.585057,1.017559,25.383533
7340,48.580002,48.880001,48.400002,48.66,42.31778,38937300.0,MSFT,2015-05-01,2015.0,2015-05-01,4.0,83.998285,0.981299,24.020711
7452,47.450001,47.540001,46.919998,47.110001,41.507988,28600600.0,MSFT,2015-10-09,2015.0,2015-10-01,4.0,111.259873,1.008491,24.905904
7467,53.32,53.990002,52.619999,52.639999,46.380405,46619800.0,MSFT,2015-10-30,2015.0,2015-10-01,4.0,69.356892,1.043313,21.467038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2096.100098,2098.899902,2069.149902,2085.5,2080.807617,1957264.0,LT.NS,2022-12-02,2022.0,2022-12-01,4.0,105.598993,1.03287,0.485883
5219,2482.75,2504.0,2444.600098,2449.350098,2443.839111,1837661.0,LT.NS,2023-07-07,2023.0,2023-07-01,4.0,72.839846,1.009349,0.404862
5234,2660.149902,2668.149902,2625.0,2649.550049,2643.588623,1325724.0,LT.NS,2023-07-28,2023.0,2023-07-01,4.0,107.527599,0.993687,0.387532
5268,2933.0,2934.949951,2900.0,2908.550049,2908.550049,1655275.0,LT.NS,2023-09-15,2023.0,2023-09-01,4.0,74.635302,0.99792,0.344637


In [ ]:
# Add a new column "profit" to df_cci_close_1
df_cci_close_1['gross_profit'] = (df_cci_close_1['Adj Close_x'] * df_cci_close_1['stocks_buy']) - 1000

df_cci_close_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,stocks_buy,gross_profit
7113,41.48,41.66,41.240002,41.48,35.373264,24060500.0,MSFT,2014-06-06,2014.0,2014-06-01,4.0,192.611211,0.993973,28.642821,13.190094
7181,46.91,47.02,46.599998,46.700001,40.073509,38244700.0,MSFT,2014-09-12,2014.0,2014-09-01,4.0,101.585057,1.017559,25.383533,17.207263
7340,48.580002,48.880001,48.400002,48.66,42.31778,38937300.0,MSFT,2015-05-01,2015.0,2015-05-01,4.0,83.998285,0.981299,24.020711,16.503151
7452,47.450001,47.540001,46.919998,47.110001,41.507988,28600600.0,MSFT,2015-10-09,2015.0,2015-10-01,4.0,111.259873,1.008491,24.905904,33.793947
7467,53.32,53.990002,52.619999,52.639999,46.380405,46619800.0,MSFT,2015-10-30,2015.0,2015-10-01,4.0,69.356892,1.043313,21.467038,-4.350092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2096.100098,2098.899902,2069.149902,2085.5,2080.807617,1957264.0,LT.NS,2022-12-02,2022.0,2022-12-01,4.0,105.598993,1.03287,0.485883,11.028926
5219,2482.75,2504.0,2444.600098,2449.350098,2443.839111,1837661.0,LT.NS,2023-07-07,2023.0,2023-07-01,4.0,72.839846,1.009349,0.404862,-10.583434
5234,2660.149902,2668.149902,2625.0,2649.550049,2643.588623,1325724.0,LT.NS,2023-07-28,2023.0,2023-07-01,4.0,107.527599,0.993687,0.387532,24.475647
5268,2933.0,2934.949951,2900.0,2908.550049,2908.550049,1655275.0,LT.NS,2023-09-15,2023.0,2023-09-01,4.0,74.635302,0.99792,0.344637,2.395213


In [ ]:
df_cci_close_1['gross_profit_cum'] = df_cci_close_1['gross_profit'].cumsum()

df_cci_close_1

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,stocks_buy,gross_profit,gross_profit_cum
7113,41.48,41.66,41.240002,41.48,35.373264,24060500.0,MSFT,2014-06-06,2014.0,2014-06-01,4.0,192.611211,0.993973,28.642821,13.190094,13.190094
7181,46.91,47.02,46.599998,46.700001,40.073509,38244700.0,MSFT,2014-09-12,2014.0,2014-09-01,4.0,101.585057,1.017559,25.383533,17.207263,30.397358
7340,48.580002,48.880001,48.400002,48.66,42.31778,38937300.0,MSFT,2015-05-01,2015.0,2015-05-01,4.0,83.998285,0.981299,24.020711,16.503151,46.900509
7452,47.450001,47.540001,46.919998,47.110001,41.507988,28600600.0,MSFT,2015-10-09,2015.0,2015-10-01,4.0,111.259873,1.008491,24.905904,33.793947,80.694456
7467,53.32,53.990002,52.619999,52.639999,46.380405,46619800.0,MSFT,2015-10-30,2015.0,2015-10-01,4.0,69.356892,1.043313,21.467038,-4.350092,76.344364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2096.100098,2098.899902,2069.149902,2085.5,2080.807617,1957264.0,LT.NS,2022-12-02,2022.0,2022-12-01,4.0,105.598993,1.03287,0.485883,11.028926,87.807083
5219,2482.75,2504.0,2444.600098,2449.350098,2443.839111,1837661.0,LT.NS,2023-07-07,2023.0,2023-07-01,4.0,72.839846,1.009349,0.404862,-10.583434,77.223649
5234,2660.149902,2668.149902,2625.0,2649.550049,2643.588623,1325724.0,LT.NS,2023-07-28,2023.0,2023-07-01,4.0,107.527599,0.993687,0.387532,24.475647,101.699296
5268,2933.0,2934.949951,2900.0,2908.550049,2908.550049,1655275.0,LT.NS,2023-09-15,2023.0,2023-09-01,4.0,74.635302,0.99792,0.344637,2.395213,104.094509


In [ ]:
df_trunc_2 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_trunc_2014_2023.parquet.brotli')

df_trunc_2 = df_trunc_2.reset_index(drop=True)

df_trunc_2

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [ ]:
# We create a smaller df for faster calculations
df_cci_2 = df_trunc_2.loc[:, ['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker','Date', 'Year', 'Month', 'Weekday', 'cci']]
df_cci_2

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014-01-02,2014,2014-01-01,3,57.700615
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014-01-03,2014,2014-01-01,4,1.373763
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014-01-06,2014,2014-01-01,0,-96.631259
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014-01-07,2014,2014-01-01,1,-83.904297
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014-01-08,2014,2014-01-01,2,-147.855135
...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023-12-22,2023,2023-12-01,4,70.767162
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023-12-26,2023,2023-12-01,1,99.598220
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023-12-27,2023,2023-12-01,2,130.401152
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023-12-28,2023,2023-12-01,3,106.774509


In [ ]:
# Filter records where CCI > 200 and it's a Friday
df_cci_filter_open_2 = df_cci_2[(df_cci_2['cci'] > 200) & (df_cci_2['Weekday'] == 4)]

df_cci_filter_open_2

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
102,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798
170,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330
329,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871
441,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828
456,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024
...,...,...,...,...,...,...,...,...,...,...,...,...
80491,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547
80638,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421
80653,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523
80687,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710


In [ ]:
df_cci_filter_open_2['stocks_buy'] = 1000 / df_cci_filter_open_2['Adj Close_x']

df_cci_filter_open_2

<ipython-input-353-87a21a478fc0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter_open_2['stocks_buy'] = 1000 / df_cci_filter_open_2['Adj Close_x']


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy
102,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821
170,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533
329,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711
441,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904
456,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80491,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883
80638,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862
80653,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532
80687,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637


In [ ]:
# Initialize empty lists to store tickers and close indices
op_tickers = []
close_index = []

# Iterate over each row in the DataFrame
for index, row in df_cci_filter_open_2.iterrows():
    # Get the ticker and add 5 to the index
    op_tickers.append(row['Ticker'])
    close_index.append(index + 5)

print(op_tickers)
print(close_index)

['MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'AMZN', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'META', 'BRK-B', 'BRK-B', 'BRK-B', 'BRK-B', 'BRK-B', 'BRK-B', 'BRK-B', 'BRK-B', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'LLY', 'AVGO

In [ ]:
# Filter df_cci_2 based on tickers and indices
df_cci_close_2 = df_cci_2[(df_cci_2.index.isin(close_index)) & (df_cci_2['Ticker'].isin(op_tickers))]

df_cci_close_2


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
107,41.480000,41.660000,41.240002,41.480000,35.373264,24060500.0,MSFT,2014-06-06,2014,2014-06-01,4,192.611211
175,46.910000,47.020000,46.599998,46.700001,40.073509,38244700.0,MSFT,2014-09-12,2014,2014-09-01,4,101.585057
334,48.580002,48.880001,48.400002,48.660000,42.317780,38937300.0,MSFT,2015-05-01,2015,2015-05-01,4,83.998285
446,47.450001,47.540001,46.919998,47.110001,41.507988,28600600.0,MSFT,2015-10-09,2015,2015-10-01,4,111.259873
461,53.320000,53.990002,52.619999,52.639999,46.380405,46619800.0,MSFT,2015-10-30,2015,2015-10-01,4,69.356892
...,...,...,...,...,...,...,...,...,...,...,...,...
80496,2096.100098,2098.899902,2069.149902,2085.500000,2080.807617,1957264.0,LT.NS,2022-12-02,2022,2022-12-01,4,105.598993
80643,2482.750000,2504.000000,2444.600098,2449.350098,2443.839111,1837661.0,LT.NS,2023-07-07,2023,2023-07-01,4,72.839846
80658,2660.149902,2668.149902,2625.000000,2649.550049,2643.588623,1325724.0,LT.NS,2023-07-28,2023,2023-07-01,4,107.527599
80692,2933.000000,2934.949951,2900.000000,2908.550049,2908.550049,1655275.0,LT.NS,2023-09-15,2023,2023-09-01,4,74.635302


In [ ]:
# Extract the 'Adj Close_x' column and rename it to 'Close_Adj_Close_x'
close_adj_close_x_2 = df_cci_close_2['Adj Close_x'].rename('Close_Adj_Close_x')

close_adj_close_x_2


107        35.373264
175        40.073509
334        42.317780
446        41.507988
461        46.380405
            ...     
80496    2080.807617
80643    2443.839111
80658    2643.588623
80692    2908.550049
80747    3378.449951
Name: Close_Adj_Close_x, Length: 460, dtype: float64

In [ ]:
# reset index for merge
df_trade_2 = df_cci_filter_open_2.reset_index(drop=True)
close_adj_close_x_2= close_adj_close_x_2.reset_index(drop=True)

In [ ]:
# Assign close_adj_close_x to a new column named 'Close_Adj_Close_x' in df_trade_2
df_trade_2['Close_Adj_Close_x'] = close_adj_close_x_2
# Display the updated DataFrame
df_trade_2


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy,Close_Adj_Close_x
0,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821,35.373264
1,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533,40.073509
2,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711,42.317780
3,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904,41.507988
4,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038,46.380405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883,2080.807617
456,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862,2443.839111
457,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532,2643.588623
458,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637,2908.550049


In [ ]:
# Create the new column 'sell_price'
df_trade_2['sell_price'] = df_trade_2['stocks_buy'] * df_trade_2['Close_Adj_Close_x']

# Display the updated DataFrame
df_trade_2


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy,Close_Adj_Close_x,sell_price
0,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821,35.373264,1013.190094
1,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533,40.073509,1017.207263
2,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711,42.317780,1016.503151
3,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904,41.507988,1033.793947
4,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038,46.380405,995.649908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883,2080.807617,1011.028926
456,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862,2443.839111,989.416566
457,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532,2643.588623,1024.475647
458,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637,2908.550049,1002.395213


In [ ]:
# Calculate the profit
df_trade_2['profit'] = df_trade_2['sell_price'] - 1000

# Display the updated DataFrame
df_trade_2


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy,Close_Adj_Close_x,sell_price,profit
0,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821,35.373264,1013.190094,13.190094
1,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533,40.073509,1017.207263,17.207263
2,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711,42.317780,1016.503151,16.503151
3,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904,41.507988,1033.793947,33.793947
4,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038,46.380405,995.649908,-4.350092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883,2080.807617,1011.028926,11.028926
456,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862,2443.839111,989.416566,-10.583434
457,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532,2643.588623,1024.475647,24.475647
458,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637,2908.550049,1002.395213,2.395213


In [ ]:
# Calculate the cumulative gross profit
df_trade_2['gross_profit'] = df_trade_2['profit'].cumsum()

# Display the updated DataFrame
df_trade_2


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,stocks_buy,Close_Adj_Close_x,sell_price,profit,gross_profit
0,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,28.642821,35.373264,1013.190094,13.190094,13.190094
1,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,25.383533,40.073509,1017.207263,17.207263,30.397358
2,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,24.020711,42.317780,1016.503151,16.503151,46.900509
3,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,24.905904,41.507988,1033.793947,33.793947,80.694456
4,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,21.467038,46.380405,995.649908,-4.350092,76.344364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,0.485883,2080.807617,1011.028926,11.028926,87.807083
456,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.404862,2443.839111,989.416566,-10.583434,77.223649
457,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,0.387532,2643.588623,1024.475647,24.475647,101.699296
458,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,0.344637,2908.550049,1002.395213,2.395213,104.094509


In [ ]:
df_trunc_3 = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Stock Analytics Zoomcamp/Module02/stocks_df_combined_trunc_2014_2023.parquet.brotli')

df_trunc_3 = df_trunc_3.reset_index(drop=True)

df_trunc_3

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [ ]:
# We create a smaller df for faster calculations
df_cci_3 = df_trunc_3.loc[:, ['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker','Date', 'Year', 'Month', 'Weekday', 'cci']]
df_cci_3

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014-01-02,2014,2014-01-01,3,57.700615
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014-01-03,2014,2014-01-01,4,1.373763
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014-01-06,2014,2014-01-01,0,-96.631259
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014-01-07,2014,2014-01-01,1,-83.904297
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014-01-08,2014,2014-01-01,2,-147.855135
...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023-12-22,2023,2023-12-01,4,70.767162
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023-12-26,2023,2023-12-01,1,99.598220
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023-12-27,2023,2023-12-01,2,130.401152
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023-12-28,2023,2023-12-01,3,106.774509


In [ ]:
df_cci_3['growth_future_5d'] = df_cci_3['Adj Close_x'].shift(-5) / df_cci_3['Adj Close_x']

df_cci_3

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014-01-02,2014,2014-01-01,3,57.700615,0.956136
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014-01-03,2014,2014-01-01,4,1.373763,0.976429
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014-01-06,2014,2014-01-01,0,-96.631259,0.968170
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014-01-07,2014,2014-01-01,1,-83.904297,0.982698
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014-01-08,2014,2014-01-01,2,-147.855135,1.027964
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023-12-22,2023,2023-12-01,4,70.767162,NaN
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023-12-26,2023,2023-12-01,1,99.598220,NaN
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023-12-27,2023,2023-12-01,2,130.401152,NaN
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023-12-28,2023,2023-12-01,3,106.774509,NaN


In [ ]:
# Filter records where CCI > 200 and it's a Friday
df_cci_filter_3 = df_cci_3[(df_cci_3['cci'] > 200) & (df_cci_3['Weekday'] == 4)]

df_cci_filter_3

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d
102,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190
170,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207
329,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503
441,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794
456,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80491,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029
80638,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417
80653,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476
80687,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395


In [ ]:
df_cci_filter_3['gross_profit'] = 1000 * df_cci_filter_3['growth_future_5d'] - 1000

df_cci_filter_3

<ipython-input-366-28c8925f77a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter_3['gross_profit'] = 1000 * df_cci_filter_3['growth_future_5d'] - 1000


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,gross_profit
102,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190,13.190094
170,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207,17.207263
329,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503,16.503151
441,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794,33.793947
456,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650,-4.350092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80491,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029,11.028926
80638,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417,-10.583434
80653,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476,24.475647
80687,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395,2.395213


In [ ]:
# Calculate the cumulative gross profit
df_cci_filter_3['gross_profit_cum'] = df_cci_filter_3['gross_profit'].cumsum()

# Display the updated DataFrame
df_cci_filter_3


<ipython-input-367-3ebb86934fa4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cci_filter_3['gross_profit_cum'] = df_cci_filter_3['gross_profit'].cumsum()


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Date,Year,Month,Weekday,cci,growth_future_5d,gross_profit,gross_profit_cum
102,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014-05-30,2014,2014-05-01,4,203.518798,1.013190,13.190094,13.190094
170,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014-09-05,2014,2014-09-01,4,257.013330,1.017207,17.207263,30.397358
329,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015-04-24,2015,2015-04-01,4,340.009871,1.016503,16.503151,46.900509
441,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015-10-02,2015,2015-10-01,4,206.125828,1.033794,33.793947,80.694456
456,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015-10-23,2015,2015-10-01,4,431.107024,0.995650,-4.350092,76.344364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80491,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022-11-25,2022,2022-11-01,4,210.797547,1.011029,11.028926,87.807083
80638,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023-06-30,2023,2023-06-01,4,219.106421,0.989417,-10.583434,77.223649
80653,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023-07-21,2023,2023-07-01,4,359.236523,1.024476,24.475647,101.699296
80687,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023-09-08,2023,2023-09-01,4,303.781710,1.002395,2.395213,104.094509


[EXPLORATORY] Question 5: Finding Your Strategy for IPOs
You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

# Answer Question 5:
# I have read a lot about IPOs. I can't have a clear strategy of what I would try to do. The most summarized information that I liked was: https://www.religareonline.com/blog/ipo-investments-tips-and-strategies/
#Maybe my strategy, given enough money, would be to do good research, diversify the portfolio well and take it as a long, very long term investment. The companies that really take off, will compensate the losses of the ones that disappeared.